# Flatiron School Phase 3 Project

Student name: **Angelo Turri**

Student pace: **self paced**

Project finish date: **?**

Instructor name: **Mark Barbour**

Blog post URL: **?**

# Stakeholder

Trying to identify pumps that need to be fixed / they want to allocate their resources wisely.

# Data Origin & Description

The data was taken from **Kaggle link here**

Descriptions of any of the original variables can be found **below in the dictionary**.

Important variables in the original dataset include:

**list here**

# Rationale & Limitations

Why are you using **method**?

What about the current problem makes linear regression suitable?

Assumptions of our model


Kind of data used

In [165]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, get_scorer_names, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statistics import mode as md
from matplotlib import pyplot as plt
from IPython.display import clear_output, display_html 
from collections import Counter
from pprint import pprint
from itertools import product
import scipy.stats as ss

In [166]:
import warnings
warnings.filterwarnings("ignore")

In [167]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [168]:
testing = pd.read_csv("tanzanian_water_wells/X_test.csv")
X = pd.read_csv("tanzanian_water_wells/X_train.csv")
y = pd.read_csv("tanzanian_water_wells/y_train.csv")['status_group'].map({'functional': 2, 'functional needs repair': 0, 'non functional': 1})

In [ ]:
idx = 

In [169]:
state = 42

# Feature Selection

There are quite a few features in this dataset (37). The descriptions of each of these features was taken from Kaggle and listed below. We will not be using all the features, for several reasons:

- Some do not correlate with the target variable (status_group), e.g., the row ID;
- Others have too many categories;
- Some variables differ massively in their values from one dataset to another;
- Others are near copies of different variables in the same dataset, making it pointless to use them.

In [170]:
desc = {'amount_tsh': 'Total static head (amount water available to waterpoint)',
                    'date_recorded': 'The date the row was entered',
                    'funder': 'Who funded the well',
                    'gps_height': 'Altitude of the well',
                    'installer': 'Organization that installed the well', 
                    'id': 'unique identifier of waterpoint',
                    'longitude': 'GPS coordinate',
                    'latitude': 'GPS coordinate',
                    'wpt_name': 'Name of the waterpoint if there is one',
                    'subvillage': 'Geographic location',
                    'region': 'Geographic location',
                    'region_code': 'Geographic location (coded)',
                    'district_code': 'Geographic location (coded)',
                    'lga': 'Geographic location',
                    'ward': 'Geographic location',
                    'population': 'Population around the well',
                    'public_meeting': 'True/False',
                    'recorded_by': 'Group entering this row of data',
                    'scheme_management': 'Who operates the waterpoint',
                    'scheme_name': 'Who operates the waterpoint',
                    'permit': 'If the waterpoint is permitted',
                    'construction_year': 'Year the waterpoint was constructed',
                    'extraction_type': 'The kind of extraction the waterpoint uses',
                    'extraction_type_group': 'The kind of extraction the waterpoint uses',
                    'extraction_type_class': 'The kind of extraction the waterpoint uses',
                    'management': 'How the waterpoint is managed',
                    'management_group': 'How the waterpoint is managed',
                    'payment': 'What the water costs',
                    'payment_type': 'What the water costs',
                    'water_quality': 'The quality of the water',
                    'quality_group': 'The quality of the water',
                    'quantity': 'The quantity of water',
                    'quantity_group': 'The quantity of water',
                    'source': 'The source of the water',
                    'source_type': 'The source of the water',
                    'source_class': 'The source of the water',
                    'waterpoint_type': 'The kind of waterpoint',
                    'waterpoint_type_group': 'The kind of waterpoint'}

### Variable inconsistency across train and test sets

We were given two datasets for analysis – one training dataset, complete with feature and target variables, and a testing dataset, which only had the feature variables (the target values were hidden).

Our goal is to accurately predict the target values in the test dataset. To do this successfully, we must use features in our model that correlate well with the target

Unfortunately, some variables will correlate well with the target in our training dataset, but not so in the test dataset. These variables are categorical, and their categories differ from one dataset to another. In the table below, you see that some categorical variables differ massively in their values from one dataset to another. Therefore, any model that uses these variables will have less success in the test dataset than in the training dataset.

It seems like the following variables should be removed from our features:

- wpt_name
- subvillage
- installer
- funder
- scheme_name
- ward
- date_recorded

In [171]:
differences = []

columns = list(X.select_dtypes(exclude=['float64', 'int64']).columns)

for col in columns:
    difference = set(list(X[col])) ^ set(list(testing[col]))
    differences.append(len(difference))
    
differences_df = pd.DataFrame({'column': list(columns), 'differences': differences}).sort_values(by=['differences'], ascending=False)
differences_df.head(10)

,column,differences
3,wpt_name,43128
5,subvillage,15120
2,installer,1584
1,funder,1403
12,scheme_name,1251
8,ward,145
0,date_recorded,51
14,extraction_type,1
23,quantity,0
21,water_quality,0


In [172]:
X = X.drop(list(differences_df['column'])[:7], axis=1)
testing = testing.drop(list(differences_df['column'])[:7], axis=1)

### Dropping variables that do not correlate with the target column

Two variables, namely "id" and "recorded_by", do not correlate with the target column (status_group).

- **id** is a unique numerical identifier for waterpoints; each waterpoint has a different identifier. An identifier such as this one cannot meaningfully correlate with the target column.

- **recorded_by** has only one value in the entire dataset, and therefore cannot correlate with the target column

In [173]:
X.id

0        69572
1         8776
2        34310
3        67743
4        19728
         ...  
59395    60739
59396    27263
59397    37057
59398    31282
59399    26348
Name: id, Length: 59400, dtype: int64

In [174]:
X.recorded_by.value_counts()

recorded_by
GeoData Consultants Ltd    59400
Name: count, dtype: int64

In [175]:
X = X.drop(['id', 'recorded_by'], axis=1)
testing = testing.drop(['id', 'recorded_by'], axis=1)

### Eliminating variables that correlate with each other

We want to eliminate colinearity. I am going to assume that the following groups of variables are strongly correlated with each other, based on the names they were given:

- **region** and **region_code**
- **scheme_management** and **scheme_name**
- **extraction_type**, **extraction_type_group** and **extraction_type_class**
- **management** and **management_group**
- **payment** and **payment_type**
- **water_quality** and **quality_group**
- **quantity** and **quantity_group**
- **source**, **source_type** and **source_class**
- **waterpoint_type** and **waterpoint_type_group**

However, I need to back it up. So I will search amongst the correlations between the categorical variables. While I'm at it, I might as well look at the correlations amongst the numerical variables too.

How do we decide between these variables? I will use the chi-squared test for indepenence to determine how strongly the variables correlate with our target variable, "status_group." Whichever of these variables more strongly correlates with status_group will be used in our models.

In [176]:
# Eliminating null values from X_train
X.scheme_management.fillna("None", inplace=True)
X.permit.fillna('Unknown', inplace=True)
X.public_meeting.fillna('Unknown', inplace=True)

# X['public_meeting'] = X['public_meeting'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['permit'] = X['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['gps_height'] = X['gps_height'].astype('float64')
# X['district_code'] = X['district_code'].astype('float64')
X['population'] = X['population'].astype('float64')
X['construction_year'] = X['construction_year'].astype('int64')
X['region_code'] = X['region_code'].astype('str')
X['district_code'] = X['district_code'].astype('str')

X_cat = X.select_dtypes(exclude=['float64', 'int64'])
X_cat = X_cat.astype('str')
X_numeric = X.select_dtypes(['float64', 'int64'])

oe = OrdinalEncoder()
oe.fit(X_cat)
X_cat = pd.DataFrame(oe.transform(X_cat), index = X_cat.index, columns = X_cat.columns)

mms = MinMaxScaler()
mms.fit(X_numeric)
X_numeric = pd.DataFrame(mms.transform(X_numeric), columns = X_numeric.columns, index = X_numeric.index)

In [177]:
# X = X.drop(['extraction_type_class', 'extraction_type', 'waterpoint_type_group', 
#             'region_code', 'payment_type', 'quantity', 'source', 'source_class', 
#             'water_quality', 'management', 'management_group'], axis=1)

# X_cat = X_cat.drop(['extraction_type_class', 'extraction_type', 'waterpoint_type_group', 
#             'region_code', 'payment_type', 'quantity', 'source', 'source_class', 
#             'water_quality', 'management', 'management_group'], axis=1)

# testing = testing.drop(['extraction_type_class', 'extraction_type', 'waterpoint_type_group', 
#             'region_code', 'payment_type', 'quantity', 'source', 'source_class', 
#             'water_quality', 'management', 'management_group'], axis=1)

# Checking for collinearity among the variables

We can check for collinearity among the numeric variables, but checking amongst the categorical variables is a waste of time.

In [178]:
# X_cat.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).head(50)

In [179]:
def cts(val):
    if (val < -0.5):
        color = 'red'
    elif (val > 0.5):
        color = 'red'
    else: 
        color = 'black'
    return 'color: %s' % color

In [180]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.concat([X_numeric, X_cat], axis=1).corr().style.applymap(cts)

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,basin,region,region_code,district_code,lga,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
amount_tsh,1.000000,0.076650,0.022134,-0.052670,0.002944,0.016288,0.067915,0.013977,-0.019866,-0.036760,-0.028303,0.010315,-0.000341,0.009852,0.024756,-0.025091,-0.017738,-0.013213,-0.008835,-0.001836,0.013623,-0.118563,0.012624,-0.023995,-0.014526,-0.014526,0.001201,-0.000711,0.034922,-0.043821,-0.042143
gps_height,0.076650,1.000000,0.149155,-0.035751,0.007237,0.135003,0.658727,-0.161768,-0.323833,-0.172466,-0.111196,0.137375,0.025325,0.082152,0.075141,-0.237886,-0.261342,-0.232483,-0.037572,0.025610,0.114219,-0.097431,0.140459,-0.092273,-0.030781,-0.030781,0.121495,0.124323,0.041968,-0.236758,-0.215989
longitude,0.022134,0.149155,1.000000,-0.425802,0.023873,0.086590,0.396732,0.216088,-0.142783,0.268935,0.089039,0.227492,-0.048005,-0.079758,0.097715,-0.027119,-0.018322,0.049411,-0.176484,-0.080633,0.015608,-0.007441,-0.051622,-0.031906,0.015785,0.015785,-0.013228,-0.047479,0.020594,-0.163355,-0.144605
latitude,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.022152,-0.245278,-0.218502,-0.025485,0.078699,-0.049026,-0.268503,0.045487,0.016553,0.073557,-0.005193,-0.000162,0.001696,0.048213,-0.056396,-0.041983,0.145438,-0.010955,0.038672,0.117193,0.117193,0.024544,0.064626,-0.040928,0.160127,0.146218
num_private,0.002944,0.007237,0.023873,0.006837,1.000000,0.003818,0.026056,0.018325,0.037351,0.033997,0.007959,-0.000843,0.002398,0.007336,0.010618,0.017254,0.014590,0.024027,-0.008308,-0.034356,-0.000894,0.008963,-0.001533,-0.006318,0.003544,0.003544,-0.007913,-0.010613,0.008769,-0.021993,-0.019882
population,0.016288,0.135003,0.086590,-0.022152,0.003818,1.000000,0.260910,0.066899,-0.004416,0.146033,-0.054335,-0.004088,0.010163,-0.035422,-0.036358,0.065397,0.060702,0.079631,-0.045170,-0.022093,0.033582,0.034478,-0.031126,0.024627,0.031487,0.031487,-0.089433,-0.090925,0.034772,0.029213,0.016292
construction_year,0.067915,0.658727,0.396732,-0.245278,0.026056,0.260910,1.000000,0.268151,-0.141681,0.376624,-0.132334,0.133283,0.032668,-0.017954,0.080284,-0.061434,-0.056986,0.010066,-0.125290,-0.074380,0.129046,-0.016861,0.050884,-0.008012,-0.008484,-0.008484,-0.007553,-0.025503,0.096427,-0.194506,-0.179239
basin,0.013977,-0.161768,0.216088,-0.218502,0.018325,0.066899,0.268151,1.000000,-0.114880,0.491692,0.009754,-0.006718,0.016602,-0.081049,0.079343,0.167450,0.189307,0.177939,-0.061139,-0.068413,-0.017788,-0.029961,-0.075529,0.044721,-0.026707,-0.026707,-0.061457,-0.074808,0.031636,0.009864,0.013518
region,-0.019866,-0.323833,-0.142783,-0.025485,0.037351,-0.004416,-0.141681,-0.114880,1.000000,0.062788,-0.071604,0.193919,-0.028418,-0.049584,-0.044192,0.217222,0.233877,0.210724,0.076552,-0.023112,-0.054115,0.056709,-0.073382,0.129883,0.034766,0.034766,-0.156225,-0.140439,-0.010899,0.279421,0.257183
region_code,-0.036760,-0.172466,0.268935,0.078699,0.033997,0.146033,0.376624,0.491692,0.062788,1.000000,-0.000508,-0.039177,0.028479,-0.147239,0.113292,0.174353,0.202180,0.256832,-0.124408,-0.131360,-0.034030,0.142661,-0.094935,0.113627,0.057630,0.057630,-0.060097,-0.074246,-0.008932,0.085874,0.087402


In [181]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

### Strong correlations

These are the strong correlations we observed among our features:

- gps_height and construction_year
- scheme_management, management and management_group
- extraction, extraction_type_group and extraction_type_class
- payment and payment_type
- quantity and quantity_group
- source and source_type
- waterpoint_type and waterpoint_type_group

Of each of these groups, we must pick only one variable to keep.

### Correlation to status_group using chi-squared test of independence

In [182]:
groups = [['gps_height', 'construction_year'], 
          ['scheme_management', 'management', 'management_group'], 
          ['extraction_type', 'extraction_type_group', 'extraction_type_class'], 
          ['payment', 'payment_type'], 
          ['quantity', 'quantity_group'], 
          ['source', 'source_type'], 
          ['waterpoint_type', 'waterpoint_type_group']]

X = pd.concat([X_numeric, X_cat], axis=1)

for group in groups:
    function = SelectKBest(score_func=chi2, k='all').fit(X[group], y)
    function_results = pd.DataFrame({'feature': function.feature_names_in_, 'score': function.scores_}).sort_values(by=['score'], ascending=False).reset_index(drop=True)
    df = function_results.sort_values(by=['score'], ascending=False)
    display(df)

,feature,score
0,gps_height,172.710476
1,construction_year,66.593012


,feature,score
0,scheme_management,310.768525
1,management,176.680250
2,management_group,30.233213


,feature,score
0,extraction_type_class,4962.445269
1,extraction_type_group,3427.761791
2,extraction_type,2638.196579


,feature,score
0,payment,866.203572
1,payment_type,462.559171


,feature,score
0,quantity,672.865229
1,quantity_group,672.865229


,feature,score
0,source_type,568.229788
1,source,458.830986


,feature,score
0,waterpoint_type,3348.517448
1,waterpoint_type_group,2540.881101


This determines which variables we will keep, and which we will eliminate. The ones we will eliminate are:

- construction_year
- management
- management_group
- extraction_type_group
- extraction_type
- payment_type
- quantity_group (this choice doesn't matter, both are perfectly correlated with each other)
- source
- waterpoint_type_group

In [183]:
X_cat = X_cat.drop(['management', 'management_group', 
        'extraction_type_group', 'extraction_type', 'payment_type', 
        'quantity_group', 'source', 'waterpoint_type_group'], axis=1)
X_cat = X_cat.astype('str')
X_cat = pd.get_dummies(X_cat).astype('int64')


X_numeric = X_numeric.drop(['construction_year'], axis=1)
scaler = MinMaxScaler()
scaler.fit(X_numeric)
X_numeric = pd.DataFrame(scaler.transform(X_numeric), columns = X_numeric.columns, index = X_numeric.index)

X = pd.concat([X_numeric, X_cat], axis=1)

# Creating Test and Train Sets

The stakeholder asks us for predictions on the test set, but we do not know what the target values for this test set are (naturally – if we had those, there would be no need for the stakeholder to ask us to do this).

In order to evaluate the performance of our models, we need to create test and train sets from our complete data.

In [184]:
#Creating a train-test-split for X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=state)

# Base Model – Logistic Regression, No Regularization, No Resampling

In [56]:
estimator = LogisticRegression()
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_test)

pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.498039,0.786594,0.724352,0.740539,0.669662,0.732501
recall,0.120038,0.650492,0.887764,0.740539,0.552765,0.740539
f1-score,0.193450,0.712098,0.797776,0.740539,0.567775,0.721309
support,1058.000000,5791.000000,8001.000000,0.740539,14850.000000,14850.000000


# Major Issue – Imbalance Target Categories

As you can see in the value counts of our target variable, the categories are extremely imbalanced. The category of "0", which means the waterpoint has a status of "functional needs repair," has way less values than 1 and 2, which mean "non functional" and "functional," respectively.

We can deal with this situation by introducing new artificial data points of the underrepresented category into our dataset, using SMOTE (Synthetic Minority Oversampling Technique). By default, this function gives each category the same number of occurrences as the most-frequent category, which in this case is "1" - "functional."

In [57]:
y_train.value_counts()

status_group
2    24258
1    17033
0     3259
Name: count, dtype: int64

In [58]:
#Resampling dataframes for model creation
smote = SMOTE(random_state=state)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [59]:
y_train_resampled.value_counts()

status_group
1    24258
2    24258
0    24258
Name: count, dtype: int64

In [60]:
estimator = LogisticRegression()
estimator.fit(X_train_resampled, y_train_resampled)
predictions = estimator.predict(X_test)

pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.241536,0.777545,0.769010,0.685724,0.596030,0.734758
recall,0.552930,0.640995,0.735658,0.685724,0.643194,0.685724
f1-score,0.336207,0.702698,0.751964,0.685724,0.596956,0.703131
support,1058.000000,5791.000000,8001.000000,0.685724,14850.000000,14850.000000


# Analysis – Oversampling Effect

If our model is to be successful, we have to predict the "functional needs repair" instances with respectable accuracy. 13% recall is inexcusable. As we can see with our over-sampling technique, we can improve our accuracy and recall in this category significantly (the f1-score improved from 0.2 to 0.33), but it will slightly decrease our metrics in other categories (functional f1-score decreased from 0.80 to 0.74).

Testing other over-sampling ratios reveals that resampling "functional needs repair" instances to about 10-15,000 instances is ideal for predictive accuracy/recall. Our F1-score for this category tops out at around 0.33-0.34.

This still isn't good, so maybe it's time to use different estimators – e.g., random forests, gradient boosting techniques, etc.

In [62]:
strategies = {str(n): {0: n} for n in [10000, 15000, 20000]}
reports = {}

for name, strategy in strategies.items():
    
    smote = SMOTE(sampling_strategy = strategy, random_state=state)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    estimator =  LogisticRegression()
    estimator.fit(X_train_resampled, y_train_resampled)
    predictions = estimator.predict(X_test)

    report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True))
    reports[name] = report
    
for name, report in reports.items():
    display(report.style.set_caption(f"resampling strategy: {name}"))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.408451,0.788292,0.731720,0.739057,0.642821,0.730750
recall,0.191871,0.648765,0.876765,0.739057,0.572467,0.739057
f1-score,0.261093,0.711755,0.797703,0.739057,0.590184,0.725955
support,1058.000000,5791.000000,8001.000000,0.739057,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.315609,0.795277,0.741729,0.724916,0.617538,0.732252
recall,0.347826,0.633915,0.840645,0.724916,0.607462,0.724916
f1-score,0.330935,0.705487,0.788095,0.724916,0.608172,0.723310
support,1058.000000,5791.000000,8001.000000,0.724916,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.268841,0.802049,0.751428,0.708956,0.607439,0.736786
recall,0.451796,0.622000,0.805899,0.708956,0.626565,0.708956
f1-score,0.337094,0.700642,0.777711,0.708956,0.605149,0.716264
support,1058.000000,5791.000000,8001.000000,0.708956,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.239443,0.807246,0.754456,0.690101,0.600382,0.738350
recall,0.519849,0.600242,0.777653,0.690101,0.632581,0.690101
f1-score,0.327869,0.688521,0.765879,0.690101,0.594090,0.704506
support,1058.000000,5791.000000,8001.000000,0.690101,14850.000000,14850.000000


# Using different estimators

In [64]:
estimators = {'Logistic Regression': LogisticRegression(solver='liblinear', fit_intercept=False, random_state=state), 
              'Decision Tree Classifier': DecisionTreeClassifier(random_state=state), 
              'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3), 
              'Bagging Classifier': BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, max_features=50, random_state=state), 
              'Random Forest': RandomForestClassifier(random_state=state), 
              'XG-Boost': XGBClassifier(random_state=state), 
              'Adaboost': AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=state), 
              'Gradient Boosted Trees': GradientBoostingClassifier(random_state=42, n_estimators=200, max_features=50, random_state=state), 
              'Extra Randomized Trees': ExtraTreesClassifier(n_estimators=100, random_state=state)}

In [88]:
resampled_datasets = {str(n): SMOTE(sampling_strategy={0: n}, random_state=state).fit_resample(X_train, y_train) for n in [3243, 0.10000, 15000, 20000, 'auto']}
resampled_datasets = [(name, key[0], key[1]) for name, key in resampled_datasets.items()]

# results_dict = {}

# for name, estimator in estimators.items():
#     results_dict[name] = [estimator, resampled_datasets]

In [100]:
results_dict = {name: None for name in estimators.keys()}

for name, estimator in estimators.items():
    
    print(f"Estimator: {name}")
    
    results = {}
    
    for ratio, ratio_X, ratio_y in resampled_datasets:
        
        print(f"Sampling ratio: {ratio}")
        
        est = estimator
        est.fit(ratio_X, ratio_y)
        predictions = est.predict(X_test)
        report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True))
        
        results[ratio] = (predictions, report)
        
    results_dict[name] = results
    clear_output(wait=True)

Estimator: Extra Randomized Trees
Sampling ratio: 10000
Sampling ratio: 15000
Sampling ratio: 20000


In [103]:
print("\n")
for name, results in results_dict.items():
    print(color.BOLD + name + color.END)
    
    for ratio, tup in results.items():
        display(tup[1].style.set_caption(f"resampled data (n = {ratio})"))
    print("\n\n")



Logistic Regression


,0,1,2,accuracy,macro avg,weighted avg
precision,0.306865,0.801093,0.742665,0.725185,0.616874,0.734401
recall,0.350662,0.632879,0.841520,0.725185,0.608353,0.725185
f1-score,0.327305,0.707119,0.789008,0.725185,0.607811,0.724180
support,1058.000000,5791.000000,8001.000000,0.725185,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.260097,0.807164,0.748656,0.705051,0.605305,0.736664
recall,0.456522,0.618719,0.800400,0.705051,0.625213,0.705051
f1-score,0.331389,0.700489,0.773664,0.705051,0.601847,0.713618
support,1058.000000,5791.000000,8001.000000,0.705051,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.231908,0.809590,0.753878,0.684512,0.598459,0.738416
recall,0.533081,0.600587,0.765279,0.684512,0.632983,0.684512
f1-score,0.323209,0.689600,0.759536,0.684512,0.590782,0.701177
support,1058.000000,5791.000000,8001.000000,0.684512,14850.000000,14850.000000





Decision Tree Classifier


...

,0,1,2,accuracy,macro avg,weighted avg
precision,0.249614,0.854874,0.760441,0.697306,0.621643,0.760872
recall,0.611531,0.604213,0.776028,0.697306,0.663924,0.697306
f1-score,0.354521,0.708013,0.768155,0.697306,0.610230,0.715232
support,1058.000000,5791.000000,8001.000000,0.697306,14850.000000,14850.000000





Extra Randomized Trees


,0,1,2,accuracy,macro avg,weighted avg
precision,0.400320,0.807824,0.810156,0.774815,0.672767,0.780048
recall,0.472590,0.773787,0.815523,0.774815,0.687300,0.774815
f1-score,0.433463,0.790439,0.812831,0.774815,0.678911,0.777071
support,1058.000000,5791.000000,8001.000000,0.774815,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.393480,0.806818,0.811694,0.772727,0.670664,0.779997
recall,0.490548,0.772405,0.810274,0.772727,0.691076,0.772727
f1-score,0.436685,0.789237,0.810983,0.772727,0.678968,0.775836
support,1058.000000,5791.000000,8001.000000,0.772727,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.374106,0.808784,0.813865,0.770572,0.665585,0.780553
recall,0.494329,0.772751,0.805524,0.770572,0.690868,0.770572
f1-score,0.425896,0.790357,0.809673,0.770572,0.675309,0.774798
support,1058.000000,5791.000000,8001.000000,0.770572,14850.000000,14850.000000


In [185]:
# Eliminating null values from testing_train
testing.scheme_management.fillna("None", inplace=True)
testing.permit.fillna('Unknown', inplace=True)
testing.public_meeting.fillna('Unknown', inplace=True)

testing['permit'] = testing['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
testing['gps_height'] = testing['gps_height'].astype('float64')
testing['population'] = testing['population'].astype('float64')
testing['construction_year'] = testing['construction_year'].astype('int64')
testing['region_code'] = testing['region_code'].astype('str')
testing['district_code'] = testing['district_code'].astype('str')

testing_cat = testing.select_dtypes(exclude=['float64', 'int64'])
testing_cat = testing_cat.astype('str')
testing_numeric = testing.select_dtypes(['float64', 'int64'])

oe = OrdinalEncoder()
oe.fit(testing_cat)
testing_cat = pd.DataFrame(oe.transform(testing_cat), index = testing_cat.index, columns = testing_cat.columns)

mms = MinMaxScaler()
mms.fit(testing_numeric)
testing_numeric = pd.DataFrame(mms.transform(testing_numeric), columns = testing_numeric.columns, index = testing_numeric.index)

In [186]:
testing_cat = testing_cat.drop(['management', 'management_group', 
        'extraction_type_group', 'extraction_type', 'payment_type', 
        'quantity_group', 'source', 'waterpoint_type_group'], axis=1)
testing_cat = testing_cat.astype('str')
testing_cat = pd.get_dummies(testing_cat).astype('int64')


testing_numeric = testing_numeric.drop(['construction_year'], axis=1)
scaler = MinMaxScaler()
scaler.fit(testing_numeric)
testing_numeric = pd.DataFrame(scaler.transform(testing_numeric), columns = testing_numeric.columns, index = testing_numeric.index)

testing = pd.concat([testing_numeric, testing_cat], axis=1)

In [187]:
smote = SMOTE(sampling_strategy={'functional needs repair':10000}, random_state=state)
X = X.drop(['region_code_26.0'], axis=1)
y = y.map({0: 'functional needs repair', 1: 'non functional', 2: 'functional'})
X_resampled, y_resampled = smote.fit_resample(X, y)
estimator = RandomForestClassifier(random_state=state)
estimator.fit(X_resampled, y_resampled)
preds = estimator.predict(testing)

In [190]:
preds

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [191]:
see = pd.read_csv("tanzanian_water_wells/Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv")

In [196]:
idx = see.id

In [198]:
official_predictions = pd.DataFrame({'id': idx, 'status_group': preds})

In [199]:
official_predictions.to_csv('tanzanian_water_wells/preds.csv', index=False)  

In [201]:
smote = SMOTE(sampling_strategy={0:10000}, random_state=state)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

improved_forest = RandomForestClassifier(n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', max_depth=100, bootstrap=True, verbose=10, random_state=state)
improved_forest.fit(X_resampled, y_resampled)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 2000
building tree 2 of 2000
building tree 3 of 2000


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s


building tree 4 of 2000
building tree 5 of 2000
building tree 6 of 2000


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.7s remaining:    0.0s


...

building tree 1268 of 2000
building tree 1269 of 2000
building tree 1270 of 2000
building tree 1271 of 2000
building tree 1272 of 2000
building tree 1273 of 2000
building tree 1274 of 2000
building tree 1275 of 2000
building tree 1276 of 2000
building tree 1277 of 2000
building tree 1278 of 2000
building tree 1279 of 2000
building tree 1280 of 2000
building tree 1281 of 2000
building tree 1282 of 2000
building tree 1283 of 2000
building tree 1284 of 2000
building tree 1285 of 2000
building tree 1286 of 2000
building tree 1287 of 2000
building tree 1288 of 2000
building tree 1289 of 2000
building tree 1290 of 2000
building tree 1291 of 2000
building tree 1292 of 2000
building tree 1293 of 2000
building tree 1294 of 2000
building tree 1295 of 2000
building tree 1296 of 2000
building tree 1297 of 2000
building tree 1298 of 2000
building tree 1299 of 2000
building tree 1300 of 2000
building tree 1301 of 2000
building tree 1302 of 2000
building tree 1303 of 2000
building tree 1304 of 2000
b

building tree 1572 of 2000
building tree 1573 of 2000
building tree 1574 of 2000
building tree 1575 of 2000
building tree 1576 of 2000
building tree 1577 of 2000
building tree 1578 of 2000
building tree 1579 of 2000
building tree 1580 of 2000
building tree 1581 of 2000
building tree 1582 of 2000
building tree 1583 of 2000
building tree 1584 of 2000
building tree 1585 of 2000
building tree 1586 of 2000
building tree 1587 of 2000
building tree 1588 of 2000
building tree 1589 of 2000
building tree 1590 of 2000
building tree 1591 of 2000
building tree 1592 of 2000
building tree 1593 of 2000
building tree 1594 of 2000
building tree 1595 of 2000
building tree 1596 of 2000
building tree 1597 of 2000
building tree 1598 of 2000
building tree 1599 of 2000
building tree 1600 of 2000
building tree 1601 of 2000
building tree 1602 of 2000
building tree 1603 of 2000
building tree 1604 of 2000
building tree 1605 of 2000
building tree 1606 of 2000
building tree 1607 of 2000
building tree 1608 of 2000
b

building tree 1878 of 2000
building tree 1879 of 2000
building tree 1880 of 2000
building tree 1881 of 2000
building tree 1882 of 2000
building tree 1883 of 2000
building tree 1884 of 2000
building tree 1885 of 2000
building tree 1886 of 2000
building tree 1887 of 2000
building tree 1888 of 2000
building tree 1889 of 2000
building tree 1890 of 2000
building tree 1891 of 2000
building tree 1892 of 2000
building tree 1893 of 2000
building tree 1894 of 2000
building tree 1895 of 2000
building tree 1896 of 2000
building tree 1897 of 2000
building tree 1898 of 2000
building tree 1899 of 2000
building tree 1900 of 2000
building tree 1901 of 2000
building tree 1902 of 2000
building tree 1903 of 2000
building tree 1904 of 2000
building tree 1905 of 2000
building tree 1906 of 2000
building tree 1907 of 2000
building tree 1908 of 2000
building tree 1909 of 2000
building tree 1910 of 2000
building tree 1911 of 2000
building tree 1912 of 2000
building tree 1913 of 2000
building tree 1914 of 2000
b

[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  3.1min finished


TypeError: ForestClassifier.predict() missing 1 required positional argument: 'X'

In [203]:
X_test = X_test.drop(['region_code_26.0'], axis=1)
improved_preds = improved_forest.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    6.4s finished


In [208]:
pd.DataFrame(classification_report(y_test.map({0: 'functional needs repair', 1: 'non functional', 2: 'functional'}), improved_preds, output_dict=True))

,functional,functional needs repair,non functional,accuracy,macro avg,weighted avg
precision,0.947432,0.825969,0.971643,0.947273,0.915015,0.947905
recall,0.968140,0.852886,0.935365,0.947273,0.918797,0.947273
f1-score,0.957674,0.839212,0.953159,0.947273,0.916682,0.947380
support,8098.000000,1074.000000,5678.000000,0.947273,14850.000000,14850.000000


In [209]:
improved_preds = improved_forest.predict(testing)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    6.2s finished


In [213]:
improved_preds = pd.DataFrame({'id': idx, 'status_group': improved_preds})
improved_preds.to_csv('tanzanian_water_wells/improved_preds.csv', index=False)  